In [3]:
import pandas as pd
import numpy as np
import re
import json
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
import tweepy
import requests
from transformers import pipeline
import schedule
import time

In [4]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
class TwitterNLPModel:
    """NLP model for analyzing tweets and identifying traffic disruptions with coordinates"""
    
    def __init__(self, twitter_bearer_token: str, google_maps_api_key: str = None):
        """
        Initialize the Twitter NLP model
        
        Args:
            twitter_bearer_token: Twitter API Bearer Token
            google_maps_api_key: Google Maps API key for geocoding (optional)
        """
        self.bearer_token = twitter_bearer_token
        self.google_maps_api_key = google_maps_api_key
        self.client = tweepy.Client(bearer_token=twitter_bearer_token)
        
        # Initialize zero-shot classification pipeline (based on your existing code)
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        
        # Define disruption labels
        self.disruption_labels = ["protest", "accident", "road closure", "traffic jam", "construction", "emergency", "normal"]
        
        # Location extraction patterns
        self.location_patterns = [
            r'\b(?:at|on|near|in|around)\s+([A-Z][a-zA-Z\s]+(?:Road|Street|Ave|Avenue|Blvd|Boulevard|Lane|Drive|Highway|Hwy|Freeway|Bridge|Tunnel))\b',
            r'\b([A-Z][a-zA-Z\s]+(?:Road|Street|Ave|Avenue|Blvd|Boulevard|Lane|Drive|Highway|Hwy|Freeway|Bridge|Tunnel))\b',
            r'#([A-Za-z0-9_]+)',
            r'\b([A-Z][a-zA-Z\s]+(?:Junction|Intersection|Circle|Square|Plaza|Area))\b'
        ]
        
        # Cache for geocoding results
        self.geocode_cache = {}
        
        # Mock coordinates for demo (Chennai locations)
        self.mock_locations = {
            'nungambakkam': (13.0569, 80.2412),
            'anna nagar': (13.0850, 80.2101),
            't nagar': (13.0418, 80.2341),
            'adyar': (13.0067, 80.2206),
            'velachery': (12.9816, 80.2209),
            'tambaram': (12.9249, 80.1000),
            'chrompet': (12.9516, 80.1462),
            'omr': (12.8956, 80.2267),
            'ecr': (12.8270, 80.2420),
            'gst road': (12.9165, 80.1315),
            'mount road': (13.0569, 80.2412),
            'anna salai': (13.0569, 80.2412),
            'chennai airport': (12.9941, 80.1709),
            'central station': (13.0836, 80.2754),
            'egmore': (13.0732, 80.2609),
            'guindy': (13.0067, 80.2206),
            'porur': (13.0382, 80.1595),
            'vadapalani': (13.0524, 80.2123),
            'koyambedu': (13.0698, 80.1947),
            'kilpauk': (13.0889, 80.2425)
        }

In [6]:
def preprocess_tweet(self, text: str) -> str:
        """Clean and preprocess tweet text"""
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        
        # Remove mentions but keep hashtags
        text = re.sub(r'@\w+', '', text)
        
        # Remove extra whitespace
        text = ' '.join(text.split())
        
        return text.strip()


In [7]:
def classify_event(self, text: str) -> Dict:
        """Classify tweet using zero-shot classification (based on your existing code)"""
        try:
            result = self.classifier(text, self.disruption_labels)
            return {
                "label": result["labels"][0],
                "confidence": result["scores"][0],
                "all_scores": dict(zip(result["labels"], result["scores"]))
            }
        except Exception as e:
            logger.error(f"Error classifying tweet: {e}")
            return {"label": "normal", "confidence": 0.0, "all_scores": {}}


In [8]:
def extract_locations(self, text: str) -> List[str]:
        """Extract location mentions from tweet text"""
        locations = []
        
        # Use regex patterns to find locations
        for pattern in self.location_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            locations.extend(matches)
        
        # Also look for city area names
        text_lower = text.lower()
        for location in self.mock_locations.keys():
            if location in text_lower:
                locations.append(location)
        
        return list(set(locations))

In [9]:
def geocode_location(self, location: str) -> Optional[Tuple[float, float]]:
        """Convert location name to coordinates"""
        if location in self.geocode_cache:
            return self.geocode_cache[location]
        
        coords = None
        location_lower = location.lower()
        
        # Check mock locations first
        for mock_loc, mock_coords in self.mock_locations.items():
            if mock_loc in location_lower or location_lower in mock_loc:
                coords = mock_coords
                break
        
        # If not found and Google Maps API is available
        if coords is None and self.google_maps_api_key:
            try:
                url = f"https://maps.googleapis.com/maps/api/geocode/json"
                params = {
                    'address': f"{location}, Chennai, India",
                    'key': self.google_maps_api_key
                }
                
                response = requests.get(url, params=params)
                data = response.json()
                
                if data['status'] == 'OK' and data['results']:
                    location_data = data['results'][0]['geometry']['location']
                    coords = (location_data['lat'], location_data['lng'])
                
            except Exception as e:
                logger.error(f"Error geocoding with Google Maps: {e}")
        
        # Default to Chennai center if not found
        if coords is None:
            coords = (13.0827, 80.2707)  # Chennai coordinates
        
        # Cache the result
        self.geocode_cache[location] = coords
        return coords

In [10]:
def fetch_tweets(self, query: str = "traffic OR accident OR road closed OR protest", 
                    max_results: int = 100) -> List[Dict]:
        """Fetch tweets from Twitter API"""
        tweets_data = []
        
        try:
            # Fetch tweets
            tweets = self.client.search_recent_tweets(
                query=f"{query} -is:retweet lang:en",
                max_results=max_results,
                tweet_fields=['created_at', 'geo', 'context_annotations', 'public_metrics']
            )
            
            if not tweets.data:
                logger.info("No tweets found")
                return tweets_data
            
            for tweet in tweets.data:
                tweets_data.append({
                    'id': tweet.id,
                    'text': tweet.text,
                    'created_at': tweet.created_at,
                    'geo': tweet.geo if hasattr(tweet, 'geo') else None
                })
        
        except Exception as e:
            logger.error(f"Error fetching tweets: {e}")
            # Return mock tweets for demo
            tweets_data = self.get_mock_tweets()
        
        return tweets_data


In [11]:
def get_mock_tweets(self) -> List[Dict]:
        """Generate mock tweets for demo purposes"""
        mock_tweets = [
            {
                'id': '1',
                'text': 'Heavy traffic jam at Nungambakkam area due to accident. Avoid the route!',
                'created_at': datetime.now(),
                'geo': None
            },
            {
                'id': '2',
                'text': 'Protest happening near Anna Nagar causing road closures #ChennaiTraffic',
                'created_at': datetime.now() - timedelta(minutes=15),
                'geo': None
            },
            {
                'id': '3',
                'text': 'Construction work on OMR causing major delays. Take alternate route',
                'created_at': datetime.now() - timedelta(minutes=30),
                'geo': None
            },
            {
                'id': '4',
                'text': 'Emergency vehicles blocking Mount Road. Traffic moving slowly',
                'created_at': datetime.now() - timedelta(minutes=45),
                'geo': None
            },
            {
                'id': '5',
                'text': 'Water logging at Velachery junction after heavy rain. Roads flooded',
                'created_at': datetime.now() - timedelta(minutes=60),
                'geo': None
            }
        ]
        
        return mock_tweets

In [12]:
def analyze_tweets(self, query: str = "traffic OR accident OR road closed OR protest") -> List[Dict]:
        """Analyze tweets and return disruption alerts with coordinates"""
        alerts = []
        
        # Fetch tweets
        tweets = self.fetch_tweets(query)
        
        logger.info(f"Analyzing {len(tweets)} tweets...")
        
        for tweet in tweets:
            try:
                # Preprocess tweet text
                clean_text = self.preprocess_tweet(tweet['text'])
                
                # Classify the event
                classification = self.classify_event(clean_text)
                
                # Only process if it's not classified as 'normal' and has good confidence
                if classification['label'] != 'normal' and classification['confidence'] > 0.5:
                    # Extract locations
                    locations = self.extract_locations(clean_text)
                    
                    if locations:
                        for location in locations:
                            coords = self.geocode_location(location)
                            if coords:
                                alert = {
                                    'timestamp': tweet['created_at'] if isinstance(tweet['created_at'], datetime) else datetime.now(),
                                    'latitude': coords[0],
                                    'longitude': coords[1],
                                    'disruption_type': classification['label'],
                                    'confidence': classification['confidence'],
                                    'location_name': location,
                                    'description': clean_text[:100] + "..." if len(clean_text) > 100 else clean_text,
                                    'tweet_id': tweet['id'],
                                    'source': 'twitter_nlp'
                                }
                                alerts.append(alert)
                    else:
                        # If no specific location found, use general Chennai coordinates
                        alert = {
                            'timestamp': tweet['created_at'] if isinstance(tweet['created_at'], datetime) else datetime.now(),
                            'latitude': 13.0827,
                            'longitude': 80.2707,
                            'disruption_type': classification['label'],
                            'confidence': classification['confidence'],
                            'location_name': 'Chennai',
                            'description': clean_text[:100] + "..." if len(clean_text) > 100 else clean_text,
                            'tweet_id': tweet['id'],
                            'source': 'twitter_nlp'
                        }
                        alerts.append(alert)
            
            except Exception as e:
                logger.error(f"Error analyzing tweet {tweet['id']}: {e}")
                continue
        
        logger.info(f"Generated {len(alerts)} disruption alerts from tweets")
        return alerts

In [13]:
def run_periodic_analysis(self, interval_minutes: int = 5):
        """Run tweet analysis every 5 minutes"""
        logger.info(f"Starting periodic tweet analysis every {interval_minutes} minutes")
        
        def analyze_job():
            try:
                alerts = self.analyze_tweets()
                
                # Save alerts to file
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"twitter_alerts_{timestamp}.json"
                
                with open(filename, 'w') as f:
                    json.dump(alerts, f, indent=2, default=str)
                
                logger.info(f"Saved {len(alerts)} alerts to {filename}")
                
                # Print summary
                print(f"\n=== Twitter NLP Analysis Results ===")
                print(f"Timestamp: {datetime.now()}")
                print(f"Total alerts: {len(alerts)}")
                
                for alert in alerts[:5]:  # Show first 5 alerts
                    print(f"\nAlert:")
                    print(f"  Location: {alert['location_name']} ({alert['latitude']:.4f}, {alert['longitude']:.4f})")
                    print(f"  Type: {alert['disruption_type']}")
                    print(f"  Confidence: {alert['confidence']:.2f}")
                    print(f"  Description: {alert['description']}")
                
            except Exception as e:
                logger.error(f"Error in periodic analysis: {e}")
        
        # Schedule the job
        schedule.every(interval_minutes).minutes.do(analyze_job)
        
        # Run once immediately
        analyze_job()
        
        # Keep running
        while True:
            schedule.run_pending()
            time.sleep(1)

In [14]:
if __name__ == "__main__":
    # Initialize the model
    # Note: You'll need to provide your Twitter Bearer Token
    TWITTER_BEARER_TOKEN = "your_twitter_bearer_token_here"
    GOOGLE_MAPS_API_KEY = "your_google_maps_api_key_here"  # Optional
    
    # For demo purposes, we'll use mock data
    try:
        model = TwitterNLPModel(TWITTER_BEARER_TOKEN, GOOGLE_MAPS_API_KEY)
        
        # Run one-time analysis
        alerts = model.analyze_tweets()
        
        # Display results
        print(f"\n=== Twitter NLP Model Results ===")
        print(f"Total alerts generated: {len(alerts)}")
        
        for alert in alerts:
            print(f"\nAlert:")
            print(f"  Location: {alert['location_name']} ({alert['latitude']:.4f}, {alert['longitude']:.4f})")
            print(f"  Type: {alert['disruption_type']}")
            print(f"  Confidence: {alert['confidence']:.2f}")
            print(f"  Description: {alert['description']}")
            print(f"  Timestamp: {alert['timestamp']}")
        
        # Uncomment to run periodic analysis
        # model.run_periodic_analysis(interval_minutes=5)
        
    except Exception as e:
        logger.error(f"Error running Twitter NLP model: {e}")

ERROR:__main__:Error running Twitter NLP model: 'TwitterNLPModel' object has no attribute 'analyze_tweets'


In [15]:
###############################################

In [17]:
import re
import json
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
from transformers import pipeline
import folium

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
class TwitterNLPModel:
    def __init__(self):
        # Initialize zero-shot classification model
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        self.disruption_labels = ["protest", "accident", "road closure", "traffic jam", "construction", "emergency", "normal"]

        # Predefined Chennai locations
        self.mock_locations = {
            'nungambakkam': (13.0569, 80.2412),
            'anna nagar': (13.0850, 80.2101),
            't nagar': (13.0418, 80.2341),
            'adyar': (13.0067, 80.2206),
            'velachery': (12.9816, 80.2209),
            'tambaram': (12.9249, 80.1000),
            'chrompet': (12.9516, 80.1462),
            'omr': (12.8956, 80.2267),
            'ecr': (12.8270, 80.2420),
            'mount road': (13.0569, 80.2412),
            'velachery junction': (12.9800, 80.2210)
        }

    def preprocess_tweet(self, text: str) -> str:
        text = re.sub(r"http\S+|www\S+|https\S+", '', text)
        text = re.sub(r"@\w+", '', text)
        return ' '.join(text.split())

    def classify_event(self, text: str) -> Dict:
        try:
            result = self.classifier(text, self.disruption_labels)
            return {
                "label": result["labels"][0],
                "confidence": result["scores"][0],
            }
        except Exception as e:
            logger.error(f"Classification error: {e}")
            return {"label": "normal", "confidence": 0.0}

    def extract_location(self, text: str) -> Optional[Tuple[str, Tuple[float, float]]]:
        text = text.lower()
        for loc in self.mock_locations:
            if loc in text:
                return loc.title(), self.mock_locations[loc]
        return None

    def get_mock_tweets(self) -> List[Dict]:
        return [
            {
                'id': '1',
                'text': 'Heavy traffic jam at Nungambakkam area due to accident. Avoid the route!',
                'created_at': datetime.now()
            },
            {
                'id': '2',
                'text': 'Protest happening near Anna Nagar causing road closures #ChennaiTraffic',
                'created_at': datetime.now() - timedelta(minutes=15)
            },
            {
                'id': '3',
                'text': 'Construction work on OMR causing major delays. Take alternate route',
                'created_at': datetime.now() - timedelta(minutes=30)
            },
            {
                'id': '4',
                'text': 'Emergency vehicles blocking Mount Road. Traffic moving slowly',
                'created_at': datetime.now() - timedelta(minutes=45)
            },
            {
                'id': '5',
                'text': 'Water logging at Velachery junction after heavy rain. Roads flooded',
                'created_at': datetime.now() - timedelta(minutes=60)
            }
        ]

    def analyze_tweets(self) -> List[Dict]:
        tweets = self.get_mock_tweets()
        alerts = []

        for tweet in tweets:
            clean_text = self.preprocess_tweet(tweet['text'])
            classification = self.classify_event(clean_text)
            if classification['label'] != 'normal' and classification['confidence'] > 0.5:
                loc_result = self.extract_location(clean_text)
                if loc_result:
                    location_name, coords = loc_result
                else:
                    location_name = "Chennai"
                    coords = (13.0827, 80.2707)

                alerts.append({
                    'timestamp': tweet['created_at'],
                    'latitude': coords[0],
                    'longitude': coords[1],
                    'disruption_type': classification['label'],
                    'confidence': classification['confidence'],
                    'location_name': location_name,
                    'description': clean_text,
                    'tweet_id': tweet['id'],
                })

        return alerts

    def render_map(self, alerts: List[Dict], filename: str = "traffic_map.html"):
        base_map = folium.Map(location=[13.0827, 80.2707], zoom_start=11)

        for alert in alerts:
            folium.Marker(
                location=[alert['latitude'], alert['longitude']],
                popup=folium.Popup(
                    f"<b>{alert['disruption_type'].title()}</b><br>"
                    f"<b>Location:</b> {alert['location_name']}<br>"
                    f"<b>Confidence:</b> {alert['confidence']:.2f}<br>"
                    f"<b>Description:</b> {alert['description']}",
                    max_width=300
                ),
                tooltip=alert['disruption_type'].title(),
                icon=folium.Icon(color="red" if alert['confidence'] > 0.7 else "orange")
            ).add_to(base_map)

        base_map.save(filename)
        logger.info(f"Map saved to {filename}")

In [ ]:
if __name__ == "__main__":
    model = TwitterNLPModel()
    alerts = model.analyze_tweets()

    print("\n=== Twitter NLP Model Alerts ===")
    for alert in alerts:
        print(f"\nAlert:")
        print(f"  Location: {alert['location_name']} ({alert['latitude']:.4f}, {alert['longitude']:.4f})")
        print(f"  Type: {alert['disruption_type']}")
        print(f"  Confidence: {alert['confidence']:.2f}")
        print(f"  Description: {alert['description']}")
        print(f"  Timestamp: {alert['timestamp']}")

    model.render_map(alerts, filename="twitter_alerts_demo_map.html")

INFO:__main__:Map saved to twitter_alerts_demo_map.html



=== Twitter NLP Model Alerts ===

Alert:
  Location: Anna Nagar (13.0850, 80.2101)
  Type: protest
  Confidence: 0.71
  Description: Protest happening near Anna Nagar causing road closures #ChennaiTraffic
  Timestamp: 2025-07-11 14:30:52.395010

Alert:
  Location: Omr (12.8956, 80.2267)
  Type: construction
  Confidence: 0.77
  Description: Construction work on OMR causing major delays. Take alternate route
  Timestamp: 2025-07-11 14:15:52.395010

Alert:
  Location: Mount Road (13.0569, 80.2412)
  Type: road closure
  Confidence: 0.52
  Description: Emergency vehicles blocking Mount Road. Traffic moving slowly
  Timestamp: 2025-07-11 14:00:52.395010


In [31]:
import re
import json
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
from transformers import pipeline
import folium
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
class TwitterNLPModel:
    def __init__(self):
        # Initialize zero-shot classification model
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        self.disruption_labels = ["protest", "accident", "road closure", "traffic jam", "construction", "emergency", "normal"]

        # Predefined Chennai locations
        self.mock_locations = {
            'nungambakkam': (13.0569, 80.2412),
            'anna nagar': (13.0850, 80.2101),
            't nagar': (13.0418, 80.2341),
            'adyar': (13.0067, 80.2206),
            'velachery': (12.9816, 80.2209),
            'tambaram': (12.9249, 80.1000),
            'chrompet': (12.9516, 80.1462),
            'omr': (12.8956, 80.2267),
            'ecr': (12.8270, 80.2420),
            'mount road': (13.0569, 80.2412),
            'velachery junction': (12.9800, 80.2210)
        }

    def preprocess_tweet(self, text: str) -> str:
        text = re.sub(r"http\S+|www\S+|https\S+", '', text)
        text = re.sub(r"@\w+", '', text)
        return ' '.join(text.split())

    def classify_event(self, text: str) -> Dict:
        try:
            result = self.classifier(text, self.disruption_labels)
            return {
                "label": result["labels"][0],
                "confidence": result["scores"][0],
            }
        except Exception as e:
            logger.error(f"Classification error: {e}")
            return {"label": "normal", "confidence": 0.0}

    def extract_location(self, text: str) -> Optional[Tuple[str, Tuple[float, float]]]:
        text = text.lower()
        for loc in self.mock_locations:
            if loc in text:
                return loc.title(), self.mock_locations[loc]
        return None

    def get_mock_tweets(self) -> List[Dict]:
        return [
            {
                'id': '1',
                'text': 'Heavy traffic jam at Nungambakkam area due to accident. Avoid the route!',
                'created_at': datetime.now()
            },
            {
                'id': '2',
                'text': 'Protest happening near Anna Nagar causing road closures #ChennaiTraffic',
                'created_at': datetime.now() - timedelta(minutes=15)
            },
            {
                'id': '3',
                'text': 'Construction work on OMR causing major delays. Take alternate route',
                'created_at': datetime.now() - timedelta(minutes=30)
            },
            {
                'id': '4',
                'text': 'Emergency vehicles blocking Mount Road. Traffic moving slowly',
                'created_at': datetime.now() - timedelta(minutes=45)
            },
            {
                'id': '5',
                'text': 'Water logging at Velachery junction after heavy rain. Roads flooded',
                'created_at': datetime.now() - timedelta(minutes=60)
            }
        ]

    def analyze_tweets(self) -> List[Dict]:
        tweets = self.get_mock_tweets()
        alerts = []

        for tweet in tweets:
            clean_text = self.preprocess_tweet(tweet['text'])
            classification = self.classify_event(clean_text)
            if classification['label'] != 'normal' and classification['confidence'] > 0.5:
                loc_result = self.extract_location(clean_text)
                if loc_result:
                    location_name, coords = loc_result
                else:
                    location_name = "Chennai"
                    coords = (13.0827, 80.2707)

                alerts.append({
                    'timestamp': tweet['created_at'],
                    'latitude': coords[0],
                    'longitude': coords[1],
                    'disruption_type': classification['label'],
                    'confidence': classification['confidence'],
                    'location_name': location_name,
                    'description': clean_text,
                    'tweet_id': tweet['id'],
                })

        return alerts

    def render_map(self, alerts: List[Dict], filename: str = "traffic_map.html"):
        base_map = folium.Map(location=[13.0827, 80.2707], zoom_start=11)

        for alert in alerts:
            folium.Marker(
                location=[alert['latitude'], alert['longitude']],
                popup=folium.Popup(
                    f"<b>{alert['disruption_type'].title()}</b><br>"
                    f"<b>Location:</b> {alert['location_name']}<br>"
                    f"<b>Confidence:</b> {alert['confidence']:.2f}<br>"
                    f"<b>Description:</b> {alert['description']}",
                    max_width=300
                ),
                tooltip=alert['disruption_type'].title(),
                icon=folium.Icon(color="red" if alert['confidence'] > 0.7 else "orange")
            ).add_to(base_map)

        base_map.save(filename)
        logger.info(f"Map saved to {filename}")
import json
def save_alerts_to_json(alerts: List[Dict], filename: str = "twitter_alerts_output.json"):
    with open(filename, "w") as f:
        json.dump(alerts, f, indent=2, default=str)

if __name__ == "__main__":
    model = TwitterNLPModel()
    alerts = model.analyze_tweets()

    print("\n=== Twitter NLP Model Alerts ===")
    for alert in alerts:
        print(f"\nAlert:")
        print(f"  Location: {alert['location_name']} ({alert['latitude']:.4f}, {alert['longitude']:.4f})")
        print(f"  Type: {alert['disruption_type']}")
        print(f"  Confidence: {alert['confidence']:.2f}")
        print(f"  Description: {alert['description']}")
        print(f"  Timestamp: {alert['timestamp']}")

    # Render the map
    model.render_map(alerts, filename="twitter_alerts_demo_map.html")

    save_alerts_to_json(alerts)

    # ✅ Optional: return alerts if running in a larger application
    # Just print to console for now, since __main__ doesn’t return
    print("\n✅ Alerts saved to 'twitter_alerts_output.json'")
    


INFO:__main__:Map saved to twitter_alerts_demo_map.html



=== Twitter NLP Model Alerts ===

Alert:
  Location: Anna Nagar (13.0850, 80.2101)
  Type: protest
  Confidence: 0.71
  Description: Protest happening near Anna Nagar causing road closures #ChennaiTraffic
  Timestamp: 2025-07-11 14:47:17.050082

Alert:
  Location: Omr (12.8956, 80.2267)
  Type: construction
  Confidence: 0.77
  Description: Construction work on OMR causing major delays. Take alternate route
  Timestamp: 2025-07-11 14:32:17.050082

Alert:
  Location: Mount Road (13.0569, 80.2412)
  Type: road closure
  Confidence: 0.52
  Description: Emergency vehicles blocking Mount Road. Traffic moving slowly
  Timestamp: 2025-07-11 14:17:17.050082

✅ Alerts saved to 'twitter_alerts_output.json'
